In [1]:
# Import packages
import osgeo
from os.path import join as pjoin
import pandas as pd
import numpy as np
import xarray as xr
import xrspatial as xrs
import rioxarray

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

import pystac
from pystac_client import Client
import pystac.item_collection as pyic
import planetary_computer
import requests
import stackstac

import rich.table
import dask.diagnostics

scratch_dir = './Project/scratch' 

from pystac_client import Client
import planetary_computer as pc

In [2]:
# Define your area of interest
aoi_EMV = { #aoi Everest and Makalu Valley Polygon
   "type": "Polygon",
   "coordinates": [
     [
       [86.63380152490207, 28.56507505979343],
       [87.74001280776696, 28.479331576213163],
       [87.7789385926358, 28.213296130511523],
       [87.61479871540345, 27.906713498710847],
       [87.61479871540345, 27.906713498710847],
       [87.04816717665705, 27.239072299890207],
       [87.04816717665705, 27.239072299890207],
       [28.54835734379745, 86.06605505495027],
       [86.63380152490207, 28.56507505979343]
     ]
   ]
    }

In [2]:
# Define your area of interest
aoi_BMV = { #aoi Everest and Barun/Makalu national park polygon
  "type": "Polygon",
  "coordinates": [
    [
        [86.704525, 27.552301],
        [87.181495, 27.970180],
        [87.276902, 27.567789],
        [86.704525, 27.567789],
        [86.704525, 27.552301]
    ]
  ]
}

In [3]:
daterange = [
    #{"interval": ["1983-09-01T00:00:00Z", "1983-11-01T00:00:00Z"]}, geen data
    #{"interval": ["1986-09-01T00:00:00Z", "1986-11-01T00:00:00Z"]}, geen data
    {"interval": ["1989-08-01T00:00:00Z", "1989-11-01T00:00:00Z"]},
    #{"interval": ["1992-08-01T00:00:00Z", "1992-11-01T00:00:00Z"]},
    {"interval": ["1995-08-01T00:00:00Z", "1995-11-01T00:00:00Z"]},
    {"interval": ["1998-08-01T00:00:00Z", "1998-11-01T00:00:00Z"]},
    #{"interval": ["2001-08-01T00:00:00Z", "2001-11-01T00:00:00Z"]},
    {"interval": ["2004-08-01T00:00:00Z", "2004-11-01T00:00:00Z"]},
    #{"interval": ["2007-08-01T00:00:00Z", "2007-11-01T00:00:00Z"]},
    {"interval": ["2010-08-01T00:00:00Z", "2010-11-01T00:00:00Z"]},
    #{"interval": ["2013-08-01T00:00:00Z", "2013-11-01T00:00:00Z"]},
    {"interval": ["2016-08-01T00:00:00Z", "2016-11-01T00:00:00Z"]},
    #{"interval": ["2019-08-01T00:00:00Z", "2019-11-01T00:00:00Z"]},
    {"interval": ["2022-08-01T00:00:00Z", "2022-11-01T00:00:00Z"]}]

def importdata(aoi, daterange):
    catalog = Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
        )
    # Define your search with CQL2 syntax
    search = catalog.search(filter_lang="cql2-json", filter={
        "op": "and",
        "args": [{"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
                {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange[0]]},
                {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]}
                ]
        }
    )


    first_item = next(search.get_items())
    pc.sign_item(first_item).assets

    charts = search.get_all_items()
    print(daterange[0],len(charts))
               
        #########################
    for t in daterange[1:]:
         # Search against the Planetary Computer STAC API
        catalog = Client.open(
            "https://planetarycomputer.microsoft.com/api/stac/v1")
        # Define your search with CQL2 syntax
        search = catalog.search(filter_lang="cql2-json", filter={
            "op": "and",
                "args": [
                    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
                    {"op": "anyinteracts", "args": [{"property": "datetime"}, t]},
                    {"op": "=", "args": [{"property": "collection"}, "landsat-c2-l2"]},
                    # data cant process more than k-amount of bands so cloud coverage can be set to almost 0
                    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 2]} 
                ]
            }  
        )

        first_item = next(search.get_items())
        pc.sign_item(first_item).assets

        items = search.get_all_items()
        print(t,'items: ',len(items))
        charts = charts+items
        

    print('Length total item set:',len(charts)) 
    return charts    

charts = importdata(aoi_EMV, daterange)

{'interval': ['1989-08-01T00:00:00Z', '1989-11-01T00:00:00Z']} 20
{'interval': ['1995-08-01T00:00:00Z', '1995-11-01T00:00:00Z']} items:  28


APIError: The request exceeded the maximum allowed time, please try again. If the issue persists, please contact planetarycomputer@microsoft.com.

Debug information for support: 0D8sqZAAAAAByR8JhAfBdRr0J1PMp1AC1UEFSMjAxMDMxMDExMDE3ADkyN2FiZmE2LTE5ZjYtNGFmMS1hMDlkLWM5NTlkOWExZTY0NA==

In [32]:
ds = stackstac.stack(planetary_computer.sign(charts), epsg=6207)
ds

<xarray.DataArray 'stackstac-781328541356202d18b0d7a3d3e090b2' (time: 28,
                                                                band: 22,
                                                                y: 13006,
                                                                x: 9247)>
dask.array<fetch_raster_window, shape=(28, 22, 13006, 9247), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/31)
  * time                         (time) datetime64[ns] 1992-08-13T04:04:46.74...
    id                           (time) <U31 'LT05_L2SP_140041_19920813_02_T1...
  * band                         (band) <U13 'qa' 'red' ... 'qa_aerosol'
  * x                            (x) float64 85.53 85.53 85.53 ... 88.39 88.39
  * y                            (y) float64 29.92 29.92 29.92 ... 26.37 26.37
    platform                     (time) <U9 'landsat-5' ... 'landsat-9'
    ...                           ...
    title                        (band) object 'Surface Temperature Quality A...
    classification:bitfields     (band) object None ... [{'name': 'fill', 'le...
    common_name                  (band) object None None None ... 'coastal' None
    center_wavelength            (band) object None None None ... 10.9 0.44 None
    full_width_half_max          (band) object None None None ... 0.59 0.02 None
    epsg                         int32 6207
Attributes:
    spec:           RasterSpec(epsg=6207, bounds=(85.5302511826651, 26.365294...
    crs:            epsg:6207
    transform:      | 0.00, 0.00, 85.53|\n| 0.00,-0.00, 29.92|\n| 0.00, 0.00,...
    resolution_xy:  (0.00030960840672378254, 0.00027304006158956895)

In [5]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in charts[0].assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                      ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                      │
│ ang              │ Angle Coefficients File                                          │
│ red              │ Red Band                                                         │
│ blue             │ Blue Band                                                        │
│ drad             │ Downwelled Radiance Band                                         │
│ emis             │ Emissivity Band                                                  │
│ emsd             │ Emissivity Standard Deviation Band                               │
│ lwir             │ Surface Temperature Band                                         │
│ trad             │ Thermal Radiance Band                                            │
│ urad             │ Upwelled Radiance Band                                           │
│ atran            │ Atmospheric Transmittance Band                                   │
│ cdist            │ Cloud Distance Band                                              │
│ green            │ Green Band                                                       │
│ nir08            │ Near Infrared Band 0.8                                           │
│ swir16           │ Short-wave Infrared Band 1.6                                     │
│ swir22           │ Short-wave Infrared Band 2.2                                     │
│ mtl.txt          │ Product Metadata File (txt)                                      │
│ mtl.xml          │ Product Metadata File (xml)                                      │
│ cloud_qa         │ Cloud Quality Assessment Band                                    │
│ mtl.json         │ Product Metadata File (json)                                     │
│ qa_pixel         │ Pixel Quality Assessment Band                                    │
│ qa_radsat        │ Radiometric Saturation and Dropped Pixel Quality Assessment Band │
│ atmos_opacity    │ Atmospheric Opacity Band                                         │
│ tilejson         │ TileJSON with default rendering                                  │
│ rendered_preview │ Rendered preview                                                 │
└──────────────────┴──────────────────────────────────────────────────────────────────┘

In [33]:
ds2 = ds.sel(band = ['red', 'blue', 'green', 'nir08', 'swir16', 'swir22', 'lwir'])
ds2

<xarray.DataArray 'stackstac-781328541356202d18b0d7a3d3e090b2' (time: 28,
                                                                band: 7,
                                                                y: 13006,
                                                                x: 9247)>
dask.array<getitem, shape=(28, 7, 13006, 9247), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/31)
  * time                         (time) datetime64[ns] 1992-08-13T04:04:46.74...
    id                           (time) <U31 'LT05_L2SP_140041_19920813_02_T1...
  * band                         (band) <U13 'red' 'blue' ... 'swir22' 'lwir'
  * x                            (x) float64 85.53 85.53 85.53 ... 88.39 88.39
  * y                            (y) float64 29.92 29.92 29.92 ... 26.37 26.37
    platform                     (time) <U9 'landsat-5' ... 'landsat-9'
    ...                           ...
    title                        (band) object 'Red Band' ... 'Surface Temper...
    classification:bitfields     (band) object None None None ... None None None
    common_name                  (band) object None None None ... None None None
    center_wavelength            (band) object None None None ... None None None
    full_width_half_max          (band) object None None None ... None None None
    epsg                         int32 6207
Attributes:
    spec:           RasterSpec(epsg=6207, bounds=(85.5302511826651, 26.365294...
    crs:            epsg:6207
    transform:      | 0.00, 0.00, 85.53|\n| 0.00,-0.00, 29.92|\n| 0.00, 0.00,...
    resolution_xy:  (0.00030960840672378254, 0.00027304006158956895)

In [34]:
from IPython.display import Image

Image(url=charts[0].assets["rendered_preview"].href, width=500)

In [ ]:
with dask.diagnostics.ProgressBar():
    ts = ds2.compute()

ts.rio.to_raster(pjoin(scratch_dir, 'ts.tif'), compress='LZW')